In [1]:
import numpy as np
import cv2 as cv

In [2]:
def perform_guided_matching(kp1, kp2, des1, des2, H_coarse, radius_in_pixels=10):
    # kp1 and kp2 are both lists of cv.KeyPoint objects.

    # Normally, we need des1 and des2 to match the keypoints.
    # But for guided matching, we have an initial homography estimation which is coarsely true.
    # For very coarse homographies, radius_in_pixels should be large.

    # Cross check verimlilik icin iyi degil, pek anlamli da degil. Istenirse belki ratio test eklenebilir.

    def filter_pts(pts, pt, radius_in_pixels):
        # pts is numpy array of shape (N, 2)
        # pt is numpy array of shape (2,)
        # radius_in_pixels is a scalar.
        # Returns a list of indices of pts that are within radius_in_pixels of pt.
        idx_list = []
        for idx, pt_candidate in enumerate(pts):
            if np.linalg.norm(pt_candidate - pt) <= radius_in_pixels:
                idx_list.append(idx)
        return idx_list


    def match_pts(descriptor, des2, pt_idx_list):
        # descriptor is numpy array of shape (K,)
        # des2 is numpy array of shape (N, K)
        # pt_idx_list is a list of indices of des2.
        # Returns the index of the best match in pt_idx_list.
        distance_closest = np.inf
        idx_closest = None
        for idx in pt_idx_list:
            distance = np.linalg.norm(descriptor - des2[idx])
            if distance < distance_closest:
                distance_closest = distance
                idx_closest = idx

        return idx_closest, distance_closest

    
    pts1 = np.array([kp.pt for kp in kp1])
    pts2 = np.array([kp.pt for kp in kp2])

    # Transform pts1 to pts1_transformed using H_coarse.
    pts1_transformed = cv.perspectiveTransform(pts1.reshape(-1, 1, 2), H_coarse).reshape(-1, 2)

    # For each point in pts1_transformed, find the indices of points in pts2 that are within radius_in_pixels pixels.

    # TODO: Verimlilik icin burada aslinda quadtree falan kullanilmali.
    pt_idx_list_list = []
    for pt1_transformed in pts1_transformed:
        pt_idx_list = filter_pts(pts2, pt1_transformed, radius_in_pixels)
        pt_idx_list_list.append(pt_idx_list)
    
    matches = []
    for idx, pt_idx_list in enumerate(pt_idx_list_list):
        if len(pt_idx_list) == 0:
            continue

        idx_closest, distance_closest = match_pts(des1[idx], des2, pt_idx_list)
        assert idx_closest is not None
        match = cv.DMatch(idx, idx_closest, distance_closest)
        matches.append(match)
    
    return matches


# TODO: AdaLAM'e benzer sekilde angle ve size bilgilerini de kullanabiliriz.
# Ama findHomography'de bu bilgi kullanilmiyor. Neden? (2 noktadan homografi bulma makalesinde kullaniliyor!)
# Gerci match etme kismi findHomography'dense AdaLAM'e daha cok benziyor. Ama orada bule opsiyoneldi sanki bu kullanim.

# TODO: Daha da onemlisi su: Homografinin her yerinde ayni miktarda hata beklenmez.
# Homografinin genisletme yaptigi yerlerde daha cok hata beklenir.
# Dolayisiyla sabit bir radius_in_pixels olmaz.
# Bizim ilk imgede o noktadaki bir radius_in_pixels yaricapli cemberimiz karsida gittigi yerde ne kadar boyutta oluyorsa o kadarlik bir alanda arama yapmak lazim.
# Tabii aslinda cember cembere gitmiyor, elips falan oluyor ama boyle aramasi daha kolay. (Belki oyle dusunmek lazim?)
# Bu olayin keypointlerin sizelariyla bir alakasi yok! Onunla karistirma. 
# Beklenen hata miktarinin imgenin her yerinde farkli olmasiyla alakasi var.

# Kabaca dogru homografiler vererek bunu test edebiliriz.
# Kabaca dogru homografileri nasil bulacagiz?
# Mesela imgenin kose noktalarini gercek homografiyle tasiriz. Sonra bunu biraz bozarak homografiyi hesaplariz (4 eslesmeden). 
# Ama gercekte boyle olmaz ki!? Daraldigi yerlerde az, genisledigi yerlerde daha cok hata olmaz mi?
# Ustte de o durumdan bahsetmistim zaten.
# Su olmaz mi acaba?
# Normal algoritma calistirilir. Bulunan inlierlardan rastgele 4 tanesinden homografi hesaplanir. Kabaca dogru olmali sonuc.
# Eger ki findHomography'deki threshold yuksekse daha coarse bir homografi elde edilir.
# Gerci simdi dusunuyorum da reprojection error (symmetric?) homografinin her yerine esit muamele yapmiyor mu? 
# Adaptive degil ki. Bu da ayri bir fikir. Neden adaptive olmasin? Ama yapmak mantikli olsaydi muhtemelen simdiye kadar yapilirdi.

# Kaba homografileri nasıl bulacağımı buldum gibi. İmgeleri küçültüp algoritmaları çalıştır. Sonra imgeleri geri büyüt. 
# Bu gerçekçi olur. Ne kadar küçültürsen o kadar kaba homografi olur.

In [3]:
# Emre'ye coarse homography'den fine homography bulma challenge yaptirilabilir.
# Know-how olusturmasi lazim. Feature extractordan vb. bagimsiz veya onlar ozelinde.
# Deneyler tasarlamak ve calistirmak, hipotezler test etmek lazim.

In [4]:
from utils import *


def estimate_coarse_homography(img1_path, img2_path, img_divider=4, pts_divider=2.5):
    # Normalde boyle bir fonksiyon olmaz. Kabaca dogru sonuc bulmayi simule ediyoruz.
    # Hem img kucultulecek hem de bulunan inlier matchlerin sadece bir kismi kullanilacak.

    img1 = cv.resize(read_image(img1_path, is_grayscale=True), (0, 0), fx=1/img_divider, fy=1/img_divider)
    img2 = cv.resize(read_image(img2_path, is_grayscale=True), (0, 0), fx=1/img_divider, fy=1/img_divider)

    kp1 = detect_keypoints(img1)
    kp2 = detect_keypoints(img2)

    des1 = compute_descriptors(img1, kp1)
    des2 = compute_descriptors(img2, kp2)

    kp1 = [cv.KeyPoint(kp.pt[0] * img_divider, kp.pt[1] * img_divider, kp.size, kp.angle, kp.response, kp.octave, kp.class_id) for kp in kp1]
    kp2 = [cv.KeyPoint(kp.pt[0] * img_divider, kp.pt[1] * img_divider, kp.size, kp.angle, kp.response, kp.octave, kp.class_id) for kp in kp2]

    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)

    pts1 = np.int32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.int32([kp2[m.trainIdx].pt for m in matches])

    _, mask = cv.findHomography(pts1, pts2, method=cv.RANSAC, ransacReprojThreshold=3.0)

    masked_pts1 = pts1[mask.ravel() == 1]
    masked_pts2 = pts2[mask.ravel() == 1]

    masked_pts1 = masked_pts1[:int(len(masked_pts1) / pts_divider)]
    masked_pts2 = masked_pts2[:int(len(masked_pts2) / pts_divider)]

    H_estimated, _ = cv.findHomography(masked_pts1, masked_pts2, method=0, ransacReprojThreshold=3.0)

    return H_estimated, img1, img2


img1_no = 1
img2_no = 3

img_divider = 4
pts_divider = 2.5

img1_path = f'homography_dataset/img{img1_no}.png'
img2_path = f'homography_dataset/img{img2_no}.png'
h_path = f'homography_dataset/H{img1_no}to{img2_no}p'

H_estimated, img1, img2 = estimate_coarse_homography(img1_path, img2_path, img_divider, pts_divider)

if img1_no == img2_no:
    H_true = np.eye(3)
else:
    H_true = np.loadtxt(h_path)

print(round(average_corner_error(img1.shape[0] * img_divider, img1.shape[1] * img_divider, H_true, H_estimated), 3))

6.523


In [5]:
img1_ori = read_image(img1_path, is_grayscale=True)
img2_ori = read_image(img2_path, is_grayscale=True)

kp1_ori = detect_keypoints(img1_ori)
kp2_ori = detect_keypoints(img2_ori)

des1_ori = compute_descriptors(img1_ori, kp1_ori)
des2_ori = compute_descriptors(img2_ori, kp2_ori)

In [6]:
matches = perform_guided_matching(kp1_ori, kp2_ori, des1_ori, des2_ori, H_estimated, radius_in_pixels=10)

# img2_no=4, img_divider=4, pts_divider=2.5
# ACE 15.142

# Bu algoritmada radius_in_pixels degerinin az ya da fazla olmasi sureyi etkilemiyor.
# Verilen ornekte 3-4 dk suruyor.

# radius_in_pixels=5  -> ACE 11.839
# radius_in_pixels=10 -> ACE 1.787
# radius_in_pixels=20 -> ACE 6.511

In [7]:
pts1 = np.int32([kp1_ori[m.queryIdx].pt for m in matches])
pts2 = np.int32([kp2_ori[m.trainIdx].pt for m in matches])

H_final, _ = cv.findHomography(pts1, pts2, method=cv.RANSAC, ransacReprojThreshold=3.0)
print(round(average_corner_error(img1_ori.shape[0], img1_ori.shape[1], H_true, H_final), 3))

3.596


In [8]:
# Task 1: Make perform_guided_matching faster. For example, use a quadtree to find the nearest neighbor of each point. "timeit" the implementations.
# Süreyi etkileyen değişkenlerin farklı değerleri için sürenin grafiğini çiz ve yorumla. (x ekseni değişken değeri, y ekseni süre gibi, her iki implementasyon da çizilir aynı şekil üzerinde.)

# Task 2: Try to improve perform_guided_matching. For example, use an adaptive radius with respect to H_coarse.
# Performansı (başarıyı) etkileyen değişkenlerin farklı değerleri için performans (veya hata) grafiğini çiz ve yorumla.

# Task 3: Try to improve perform_guided_matching by making it free of the parameter "radius_in_pixels".
# Task 2'deki gibi grafik çiz.